In [1]:
import pandas as pd

# Read the CSVs into a variable
price_data = pd.read_csv('hotels-europe_price.csv')
features_data = pd.read_csv('hotels-europe_features.csv')

In [2]:
# merge the two datasets with the key 'hotel_id'
merged_hotel_data = pd.merge(price_data, features_data, on='hotel_id')

In [3]:
# convert accommodation_type and city_actual to string so that it can be filtered
merged_hotel_data['accommodation_type'] = merged_hotel_data['accommodation_type'].astype('string')
merged_hotel_data['city_actual']=merged_hotel_data['city'].astype('string')

In [4]:
# drop duplicate values for cleaning purposes
merged_hotel_data = merged_hotel_data.drop_duplicates()

In [ ]:
# filter the merged data with the specified filters
merged_hotel_data_moscow = merged_hotel_data.loc[(merged_hotel_data["city"].isin(['Vienna','Moscow']))
                                                  ]
merged_hotel_data_filtered